# RAPIDS & Scanpy Single-Cell RNA-seq Workflow on mouse PFC cells

For demonstration purposes, we use a dataset of 1.3 M brain cells with Unified Virtual Memory to oversubscribe GPU memory.

## Import requirements

In [44]:
import numpy as np
import scanpy as sc
import anndata

import time
import os, wget

import cupy as cp
import cudf

from cuml.decomposition import PCA
from cuml.manifold import TSNE
from cuml.cluster import KMeans
from cuml.preprocessing import StandardScaler

import rapids_scanpy_funcs
import utils

import warnings
warnings.filterwarnings('ignore', 'Expected ')
warnings.simplefilter('ignore')
import pandas as pd
from sh import gunzip
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

import gc
import cupy as cp
gc.collect()
cp.get_default_memory_pool().free_all_blocks()

We use the RAPIDS memory manager to enable Unified Virtual Memory management, which allows us to oversubscribe the GPU memory.

In [2]:
from rmm.allocators.cupy import rmm_cupy_allocator
import cupy
cupy.cuda.set_allocator(rmm_cupy_allocator)

## Input data

In [51]:
import calculation_tool as ct
D_R_mtx,GPCR_type_df,drug_list,GPCR_list=ct.load_parameters()
params=ct.set_parameters_for_preprocess(GPCR_list)

In [ ]:
#old 
#file_path="/temp/data/mouse_PFC/WT_PFC_adata.h5ad"
#adata = anndata.read_h5ad(file_path)

#GPCR_file_path="/temp/data/mouse_PFC/WT_PFC_GPCR_adata.h5ad"
#GPCR_adata = anndata.read_h5ad(GPCR_file_path)

In [2]:
file_path=r"/data/mouse_PFC/combined_data_processed.h5ad"
adata = anndata.read_h5ad(file_path)

GPCR_file_path=r"/data/mouse_PFC/combined_data_GPCR_df.csv"
GPCR_df = pd.read_csv(GPCR_file_path, index_col=0)
GPCR_adata=anndata.AnnData(X=GPCR_df)
GPCR_adata_norm=sc.pp.normalize_total(GPCR_adata,target_sum=1e4,inplace=False)['X']
GPCR_adata_norm_df=pd.DataFrame(GPCR_adata_norm,columns=GPCR_adata.var.index)

In [5]:
GPCR_df.sum(axis=0)

HTR1A_raw      333.0
HTR1B_raw      240.0
HTR1D_raw       11.0
HTR2A_raw     1156.0
HTR2B_raw        1.0
HTR2C_raw      782.0
HTR3A_raw      198.0
HTR4_raw        11.0
HTR5A_raw      110.0
HTR6_raw        41.0
HTR7_raw        42.0
DRD1_raw       330.0
DRD2_raw        11.0
DRD4_raw         4.0
DRD5_raw       149.0
HRH1_raw       508.0
HRH2_raw       165.0
HRH3_raw      1356.0
CHRM1_raw     1595.0
CHRM2_raw      456.0
CHRM3_raw      628.0
CHRM5_raw        3.0
ADRA1A_raw     348.0
ADRA1B_raw     348.0
ADRA2A_raw     129.0
ADRA2C_raw      84.0
ADRB1_raw      332.0
ADRB2_raw      440.0
dtype: float64

In [ ]:
dir="/temp/data/mouse_PFC"
GPCR_adata_norm_df=pd.read_csv((os.path.join(dir,"GPCR_adata_norm_df.csv")),index_col=0)
new_columns = [col.split('_')[0] for col in GPCR_adata_norm_df.columns]
GPCR_adata_norm_df.columns=new_columns


In [44]:
from tqdm import tqdm  # 追加：進捗バー用ライブラリ

def sim_inhibit_pattern(adata, GPCR_adata_norm_df, GPCR_type_df, drug_conc,n_pattern=100):
    # 1. adata.obs の "is_clz_selective" に基づき、グループ分けするためのマスクを作成
    mask = adata.obs['is_clz_selective'] == True

    # 2. GPCRのリストおよび GPCR_type_df のフィルタリング
    # "Unnamed: 0" を除外したカラムリストを作成
    GPCR_list2 = [col for col in GPCR_adata_norm_df.columns if col != "Unnamed: 0"]
    GPCR_type_df = GPCR_type_df[GPCR_type_df.receptor_name.isin(GPCR_list2)]
    Gs = GPCR_type_df[GPCR_type_df.type == "Gs"]["receptor_name"].values
    Gi = GPCR_type_df[GPCR_type_df.type == "Gi"]["receptor_name"].values

    # 3. ランダムな受容体阻害パターンを 10,000 パターン生成
    unique_patterns_set = set()
    pattern_dict = {}
    i = 0
    while len(unique_patterns_set) < n_pattern:
        random_pattern = np.random.randint(2, size=len(GPCR_list2))
        pattern_str = ''.join(map(str, random_pattern))
        if pattern_str not in unique_patterns_set:
            unique_patterns_set.add(pattern_str)
            # 各パターンは、受容体ごとに True (阻害する) / False (阻害しない) の辞書とする
            pattern_dict[f"Pattern_{i+1}"] = {gpcr: bool(val) for gpcr, val in zip(GPCR_list2, random_pattern)}
            i += 1

    # オプション：最初の5パターンを確認
    for key in list(pattern_dict.keys())[:5]:
        print(f"{key}: {pattern_dict[key]}")

    # 4. 全細胞の GPCR 発現データ（正規化済み）の DataFrame を用意
    # ※ GPCR_adata_norm_df の index と adata.obs_names が整合している前提
    all_expr = pd.DataFrame(GPCR_adata_norm_df, index=GPCR_adata_norm_df.index, columns=GPCR_list2)

    # 5. 全細胞に対する cAMP 変化（cAMP modulation）をシミュレーションする関数の定義
    def simulate_response_all(expression_df, pattern, drug_conc, Gs, Gi):
        """
        expression_df: 各細胞の受容体発現 (DataFrame, 行=細胞, 列=受容体)
        pattern: 受容体阻害パターン（辞書, receptor -> bool, True=阻害する）
        drug_list: 薬剤名のリスト
        drug_conc: 薬剤濃度（scalar）
        Gs, Gi: Gs, Gi タイプ受容体名の配列
        """
        # 阻害パターンに応じた effective Ki の設定
        # 阻害する受容体は Ki = 0.01、阻害しない受容体は Ki = 10000
        #effective_Ki = pd.Series({receptor: (0.01 if pattern[receptor] else 10000)
        #                          for receptor in expression_df.columns})
        effective_Ki = pd.Series({
        receptor: (0.01 if pattern[receptor] else 10000)
        for receptor in expression_df.columns
        })
        #print(effective_Ki)
        responses = {}
        # 各薬剤について、Gs 効果・Gi 効果を計算
        gs_effect = (expression_df[Gs].divide(1 + drug_conc / effective_Ki[Gs])).sum(axis=1)
        gi_effect = (expression_df[Gi].divide(1 + drug_conc / effective_Ki[Gi])).sum(axis=1)
        basal_cAMP = (expression_df[Gs] - expression_df[Gi]).sum(axis=1)
        cAMPmod = (gs_effect - gi_effect) - basal_cAMP
        print("cAMPmod")
        
        # 各薬剤の結果は、細胞ごとの cAMPmod の Series とする
        responses= cAMPmod
        return responses

    # 6. 各阻害パターンについて、全細胞でシミュレーションした後、clz_selective と非選択細胞間の差分を算出
    results = []
    # adata.obs のインデックスと all_expr のインデックスが一致しない可能性があるため、再インデックスする
    mask_aligned = mask.reindex(all_expr.index, fill_value=False)
    
    # tqdm を用いて進捗状況を表示
    for pattern_name, pattern in tqdm(pattern_dict.items(), total=len(pattern_dict), desc="Simulating drug responses"):
        # 全細胞でのシミュレーション結果を得る
        all_responses = simulate_response_all(all_expr, pattern, drug_conc, Gs, Gi)
        print(all_responses)
        # 各薬剤について、mask_aligned を用いて clz_selective と非選択細胞群の平均値を計算
        selective_mean = all_responses[mask_aligned].mean()
        nonselective_mean = all_responses[~mask_aligned].mean()
        diff= selective_mean - nonselective_mean
        #print(diff)
        results.append({
            'pattern_name':pattern_name,
            'pattern': pattern,
            'diff': diff
        })

    # 7. 結果を DataFrame に変換し、diff の大きい順にソート
    results_df = pd.DataFrame(results)
    results_df_sorted = results_df.sort_values(by='diff', ascending=False)

    # 上位のパターンを確認（例：上位5件）
    print(results_df_sorted.head())

    return results_df_sorted,all_responses


In [37]:
GPCR_type_df

,receptor_name,type
0,ADRA1A,Gq
1,ADRA1B,Gq
2,ADRA2A,Gi
3,ADRA2B,Gi
4,ADRA2C,Gi
5,ADRB1,Gs
6,ADRB2,Gs
7,CHRM1,Gq
8,CHRM2,Gi
9,CHRM3,Gq


In [ ]:
#import calculation_tool as ct
drug_conc=10**4
results_df_sorted,all_responses=sim_inhibit_pattern(adata,GPCR_adata_norm_df,GPCR_type_df,drug_conc)

In [ ]:
# 1. adata.obs の "is_clz_selective" に基づき、グループ分けするためのマスクを作成
mask = adata.obs['is_clz_selective'] == True

# 2. GPCRのリストおよび GPCR_type_df のフィルタリング
# "Unnamed: 0" を除外したカラムリストを作成
GPCR_list2 = [col for col in GPCR_adata_norm_df.columns if col != "Unnamed: 0"]
#GPCR_type_df = GPCR_type_df[GPCR_type_df.receptor_name.isin(GPCR_list2)]
Gs = GPCR_type_df[GPCR_type_df.type == "Gs"]["receptor_name"].values
Gi = GPCR_type_df[GPCR_type_df.type == "Gi"]["receptor_name"].values
Gs_cols = [gene + '_raw' for gene in Gs]
Gi_cols = [gene + '_raw' for gene in Gi]
# 3. ランダムな受容体阻害パターンを 10,000 パターン生成
unique_patterns_set = set()
pattern_dict = {}
i = 0
n_pattern=10
while len(unique_patterns_set) < n_pattern:
    random_pattern = np.random.randint(2, size=len(GPCR_list2))
    pattern_str = ''.join(map(str, random_pattern))
    if pattern_str not in unique_patterns_set:
        unique_patterns_set.add(pattern_str)
        # 各パターンは、受容体ごとに True (阻害する) / False (阻害しない) の辞書とする
        pattern_dict[f"Pattern_{i+1}"] = {gpcr: bool(val) for gpcr, val in zip(GPCR_list2, random_pattern)}
        i += 1

# オプション：最初の5パターンを確認
for key in list(pattern_dict.keys())[:5]:
    print(f"{key}: {pattern_dict[key]}")

# 4. 全細胞の GPCR 発現データ（正規化済み）の DataFrame を用意
# ※ GPCR_adata_norm_df の index と adata.obs_names が整合している前提
all_expr = pd.DataFrame(GPCR_adata_norm_df, index=GPCR_adata_norm_df.index, columns=GPCR_list2)

Pattern_1: {'HTR1A_raw': False, 'HTR1B_raw': False, 'HTR1D_raw': True, 'HTR2A_raw': True, 'HTR2B_raw': False, 'HTR2C_raw': False, 'HTR3A_raw': False, 'HTR4_raw': False, 'HTR5A_raw': False, 'HTR6_raw': False, 'HTR7_raw': False, 'DRD1_raw': False, 'DRD2_raw': False, 'DRD4_raw': True, 'DRD5_raw': True, 'HRH1_raw': False, 'HRH2_raw': False, 'HRH3_raw': True, 'CHRM1_raw': False, 'CHRM2_raw': True, 'CHRM3_raw': False, 'CHRM5_raw': True, 'ADRA1A_raw': True, 'ADRA1B_raw': True, 'ADRA2A_raw': False, 'ADRA2C_raw': True, 'ADRB1_raw': False, 'ADRB2_raw': False}
Pattern_2: {'HTR1A_raw': False, 'HTR1B_raw': True, 'HTR1D_raw': False, 'HTR2A_raw': True, 'HTR2B_raw': False, 'HTR2C_raw': False, 'HTR3A_raw': True, 'HTR4_raw': False, 'HTR5A_raw': True, 'HTR6_raw': False, 'HTR7_raw': False, 'DRD1_raw': False, 'DRD2_raw': True, 'DRD4_raw': False, 'DRD5_raw': True, 'HRH1_raw': False, 'HRH2_raw': False, 'HRH3_raw': False, 'CHRM1_raw': False, 'CHRM2_raw': True, 'CHRM3_raw': True, 'CHRM5_raw': False, 'ADRA1A_ra

In [56]:
def simulate_response_all(expression_df, pattern, drug_conc, Gs, Gi):
    """
    expression_df: 各細胞の受容体発現 (DataFrame, 行=細胞, 列=受容体)
    pattern: 受容体阻害パターン（辞書, receptor -> bool, True=阻害する）
    drug_list: 薬剤名のリスト
    drug_conc: 薬剤濃度（scalar）
    Gs, Gi: Gs, Gi タイプ受容体名の配列
    """
    # 阻害パターンに応じた effective Ki の設定
    # 阻害する受容体は Ki = 0.01、阻害しない受容体は Ki = 10000
    #effective_Ki = pd.Series({receptor: (0.01 if pattern[receptor] else 10000)
    #                          for receptor in expression_df.columns})
    effective_Ki = pd.Series({
    receptor: (0.01 if pattern[receptor] else 10000)
    for receptor in expression_df.columns
    })
    #print(effective_Ki)
    responses = {}
    # 各薬剤について、Gs 効果・Gi 効果を計算
    gs_effect = (expression_df[Gs_cols].divide(1 + drug_conc / effective_Ki[Gs])).sum(axis=1)
    gi_effect = (expression_df[Gi_cols].divide(1 + drug_conc / effective_Ki[Gi])).sum(axis=1)
    basal_cAMP = (expression_df[Gs_cols] - expression_df[Gi_cols]).sum(axis=1)
    cAMPmod = (gs_effect - gi_effect) - basal_cAMP
    print("cAMPmod")
    
    # 各薬剤の結果は、細胞ごとの cAMPmod の Series とする
    responses= cAMPmod
    return responses

# 6. 各阻害パターンについて、全細胞でシミュレーションした後、clz_selective と非選択細胞間の差分を算出
results = []
# adata.obs のインデックスと all_expr のインデックスが一致しない可能性があるため、再インデックスする
mask_aligned = mask.reindex(all_expr.index, fill_value=False)

In [75]:
pattern=pattern_dict['Pattern_1']
expression_df=all_expr
effective_Ki = pd.Series({
    receptor: (0.01 if pattern[receptor] else 10000)
    for receptor in expression_df.columns
    })

# expression_df に存在し、かつ effective_Ki にも存在する GPCR のみを抽出
Gs_filtered = [gene for gene in Gs if (gene + '_raw' in expression_df.columns) and (gene + '_raw' in effective_Ki.index)]
Gi_filtered = [gene for gene in Gi if (gene + '_raw' in expression_df.columns) and (gene + '_raw' in effective_Ki.index)]

# フィルタ後のリストから、expression_df の列名用リストを作成
Gs_cols = [gene + '_raw' for gene in Gs_filtered]
Gi_cols = [gene + '_raw' for gene in Gi_filtered]

In [76]:
Gs_cols

['ADRB1_raw',
 'ADRB2_raw',
 'DRD1_raw',
 'DRD5_raw',
 'HRH2_raw',
 'HTR4_raw',
 'HTR6_raw',
 'HTR7_raw']

In [77]:
drug_conc=100

#print(effective_Ki)
responses = {}
# 各薬剤について、Gs 効果・Gi 効果を計算
gs_effect = (expression_df[Gs_cols].divide(1 + drug_conc / effective_Ki[Gs_cols])).sum(axis=1)
gi_effect = (expression_df[Gi_cols].divide(1 + drug_conc / effective_Ki[Gi_cols])).sum(axis=1)
basal_cAMP = (expression_df[Gs_cols] - expression_df[Gi_cols]).sum(axis=1)
cAMPmod = (gs_effect - gi_effect) - basal_cAMP
print("cAMPmod")
print(cAMPmod.sum())
print(expression_df.shape)
print(expression_df[Gs_cols].shape)
print(Gs_cols)
print(expression_df.sum())

cAMPmod
5282437.368931888
(27469, 28)
(27469, 8)
['ADRB1_raw', 'ADRB2_raw', 'DRD1_raw', 'DRD5_raw', 'HRH2_raw', 'HTR4_raw', 'HTR6_raw', 'HTR7_raw']
HTR1A_raw     1.517544e+06
HTR1B_raw     1.065524e+06
HTR1D_raw     4.638889e+04
HTR2A_raw     4.531739e+06
HTR2B_raw     1.000000e+04
HTR2C_raw     3.113704e+06
HTR3A_raw     6.499820e+05
HTR4_raw      3.787302e+04
HTR5A_raw     5.850559e+05
HTR6_raw      1.947302e+05
HTR7_raw      2.308869e+05
DRD1_raw      1.570817e+06
DRD2_raw      5.450000e+04
DRD4_raw      2.450000e+04
DRD5_raw      7.009246e+05
HRH1_raw      2.998083e+06
HRH2_raw      8.822891e+05
HRH3_raw      6.383453e+06
CHRM1_raw     7.308867e+06
CHRM2_raw     2.154278e+06
CHRM3_raw     3.108117e+06
CHRM5_raw     2.166667e+04
ADRA1A_raw    2.233074e+06
ADRA1B_raw    1.790834e+06
ADRA2A_raw    6.905456e+05
ADRA2C_raw    4.170040e+05
ADRB1_raw     1.626040e+06
ADRB2_raw     4.121579e+06
dtype: float64


In [64]:
effective_Ki.index

Index(['HTR1A_raw', 'HTR1B_raw', 'HTR1D_raw', 'HTR2A_raw', 'HTR2B_raw',
       'HTR2C_raw', 'HTR3A_raw', 'HTR4_raw', 'HTR5A_raw', 'HTR6_raw',
       'HTR7_raw', 'DRD1_raw', 'DRD2_raw', 'DRD4_raw', 'DRD5_raw', 'HRH1_raw',
       'HRH2_raw', 'HRH3_raw', 'CHRM1_raw', 'CHRM2_raw', 'CHRM3_raw',
       'CHRM5_raw', 'ADRA1A_raw', 'ADRA1B_raw', 'ADRA2A_raw', 'ADRA2C_raw',
       'ADRB1_raw', 'ADRB2_raw'],
      dtype='object')

In [43]:
print("DataFrameの列名:", expression_df.columns.tolist())
print("Gsリスト:", Gs)

DataFrameの列名: ['HTR1A_raw', 'HTR1B_raw', 'HTR1D_raw', 'HTR2A_raw', 'HTR2B_raw', 'HTR2C_raw', 'HTR3A_raw', 'HTR4_raw', 'HTR5A_raw', 'HTR6_raw', 'HTR7_raw', 'DRD1_raw', 'DRD2_raw', 'DRD4_raw', 'DRD5_raw', 'HRH1_raw', 'HRH2_raw', 'HRH3_raw', 'CHRM1_raw', 'CHRM2_raw', 'CHRM3_raw', 'CHRM5_raw', 'ADRA1A_raw', 'ADRA1B_raw', 'ADRA2A_raw', 'ADRA2C_raw', 'ADRB1_raw', 'ADRB2_raw']
Gsリスト: ['ADRB1' 'ADRB2' 'DRD1' 'DRD5' 'HRH2' 'HTR4' 'HTR6' 'HTR7']


In [ ]:
GPCR_type_df=pd.read_csv("/temp/GPCR_df.csv",index_col=0)

In [ ]:
GPCR_adata_norm_df

In [ ]:
import numpy as np
import pandas as pd
import anndata
import scanpy as sc
import matplotlib.pyplot as plt

# 前提：以下の変数は既に定義されているものとする
# adata: シングルセル解析の AnnData オブジェクト（obs に "is_clz_selective" などが含まれる）
# GPCR_adata_norm_df: 正規化済み GPCR 発現データの DataFrame（行=細胞, 列=受容体名）
# GPCR_type_df: 受容体タイプの DataFrame（列: receptor_name, type）; type は "Gs", "Gi" 等
# drug_list: 薬剤名のリスト（例: ["drugA", "drugB", ...]）
# drug_conc: 薬剤濃度（scalar）
# ※ D_R_mtx は本コードでは使用せず、effective Ki 値によりシミュレーションする

# 1. adata.obs の "is_clz_selective" に基づき、グループ分けするためのマスクを作成
mask = adata.obs['is_clz_selective'] == True

# 2. GPCRのリストおよび GPCR_type_df のフィルタリング
GPCR_list2 = GPCR_adata_norm_df.columns
GPCR_type_df = GPCR_type_df[GPCR_type_df.receptor_name.isin(GPCR_list2)]
Gs = GPCR_type_df[GPCR_type_df.type == "Gs"]["receptor_name"].values
Gi = GPCR_type_df[GPCR_type_df.type == "Gi"]["receptor_name"].values

# 3. ランダムな受容体阻害パターンを 10,000 パターン生成
unique_patterns_set = set()
pattern_dict = {}
i = 0
while len(unique_patterns_set) < 10000:
    random_pattern = np.random.randint(2, size=len(GPCR_list2))
    pattern_str = ''.join(map(str, random_pattern))
    if pattern_str not in unique_patterns_set:
        unique_patterns_set.add(pattern_str)
        # 各パターンは、受容体ごとに True (阻害する) / False (阻害しない) の辞書とする
        pattern_dict[f"Pattern_{i+1}"] = {gpcr: bool(val) for gpcr, val in zip(GPCR_list2, random_pattern)}
        i += 1

# オプション：最初の5パターンを確認
for key in list(pattern_dict.keys())[:5]:
    print(f"{key}: {pattern_dict[key]}")

# 4. 全細胞の GPCR 発現データ（正規化済み）の DataFrame を用意
# ※ GPCR_adata_norm_df の index と adata.obs_names が整合している前提
all_expr = pd.DataFrame(GPCR_adata_norm_df, index=GPCR_adata_norm_df.index, columns=GPCR_list2)

# 5. 全細胞に対する cAMP 変化（cAMP modulation）をシミュレーションする関数の定義
def simulate_drug_response_all(expression_df, pattern, drug_list, drug_conc, Gs, Gi):
    """
    expression_df: 各細胞の受容体発現 (DataFrame, 行=細胞, 列=受容体)
    pattern: 受容体阻害パターン（辞書, receptor -> bool, True=阻害する）
    drug_list: 薬剤名のリスト
    drug_conc: 薬剤濃度（scalar）
    Gs, Gi: Gs, Gi タイプ受容体名の配列
    """
    # 阻害パターンに応じた effective Ki の設定
    # 阻害する受容体は Ki = 0.01、阻害しない受容体は Ki = 10000
    effective_Ki = pd.Series({receptor: (0.01 if pattern[receptor] else 10000)
                              for receptor in expression_df.columns})
    
    responses = {}
    for drug in drug_list:
        # 各薬剤について、Gs 効果・Gi 効果を計算
        gs_effect = (expression_df[Gs].divide(1 + drug_conc / effective_Ki[Gs])).sum(axis=1)
        gi_effect = (expression_df[Gi].divide(1 + drug_conc / effective_Ki[Gi])).sum(axis=1)
        basal_cAMP = (expression_df[Gs] - expression_df[Gi]).sum(axis=1)
        cAMPmod = (gs_effect - gi_effect) - basal_cAMP
        # 各薬剤の結果は、細胞ごとの cAMPmod の Series とする
        responses[drug] = cAMPmod
    return responses

# 6. 各阻害パターンについて、全細胞でシミュレーションした後、clz_selective と非選択細胞間の差分を算出
results = []
for pattern_name, pattern in pattern_dict.items():
    # 全細胞でのシミュレーション結果を得る
    all_responses = simulate_drug_response_all(all_expr, pattern, drug_list, drug_conc, Gs, Gi)
    
    diff = {}
    for drug in drug_list:
        # clz_selective 細胞群の平均 cAMPmod
        selective_mean = all_responses[drug][mask].mean()
        # 非選択細胞群の平均 cAMPmod
        nonselective_mean = all_responses[drug][~mask].mean()
        diff[drug] = selective_mean - nonselective_mean
    # 複数薬剤の場合、ここでは平均差を評価指標とする
    mean_diff = np.mean(list(diff.values()))
    results.append({
        'pattern': pattern_name,
        'diff_per_drug': diff,
        'mean_diff': mean_diff
    })

# 7. 結果を DataFrame に変換し、mean_diff の大きい順にソート
results_df = pd.DataFrame(results)
results_df_sorted = results_df.sort_values(by='mean_diff', ascending=False)

# 上位のパターンを確認（例：上位5件）
print(results_df_sorted.head())


In [ ]:
mask = adata.obs['is_clz_selective'] == 1

adata_clz_selective = adata[mask, :]
adata_not_clz_selective = adata[~mask, :]

GPCR_adata_clz_selective = GPCR_adata[mask, :]
GPCR_adata_not_clz_selective = GPCR_adata[~mask, :]

GPCR_list2=GPCR_adata_norm_df.columns
GPCR_type_df=GPCR_type_df[GPCR_type_df.receptor_name.isin(GPCR_list2)]

import numpy as np

# 重複を避けるために使用するセット
unique_patterns_set = set()

# 結果を保存するための辞書
pattern_dict = {}

# 1万種類の独自の活性化パターンを生成
i = 0
while len(unique_patterns_set) < 10000:
    # ランダムな活性化パターンを生成（0はFalse、1はTrueとする）
    random_pattern = np.random.randint(2, size=len(GPCR_list2))
    # パターンを文字列に変換してハッシュ可能にする
    pattern_str = ''.join(map(str, random_pattern))

    # このパターンがまだ見つかっていない場合は保存
    if pattern_str not in unique_patterns_set:
        unique_patterns_set.add(pattern_str)
        pattern_dict[f"Pattern_{i+1}"] = {gpcr: bool(val) for gpcr, val in zip(GPCR_list2, random_pattern)}
        i += 1

# 最初の5つのパターンを出力して確認
for key in list(pattern_dict.keys())[:5]:
    print(f"{key}: {pattern_dict[key]}")

# pattern_dictをデータフレームに変換
pattern_df = pd.DataFrame.from_dict(pattern_dict, orient='index').reset_index(drop=True)

clz_s=pd.DataFrame(GPCR_adata_clz_selective.X)
clz_s.columns=GPCR_list2

clz_nons=pd.DataFrame(GPCR_adata_not_clz_selective.X)
clz_nons.columns=GPCR_list2

Gs=GPCR_type_df[GPCR_type_df.type=="Gs"]["receptor_name"].values
Gi=GPCR_type_df[GPCR_type_df.type=="Gi"]["receptor_name"].values

clz_s_Gs=clz_s.loc[:,Gs].dot(pattern_df.loc[:,Gs].T)
clz_s_Gi=clz_s.loc[:,Gi].dot(pattern_df.loc[:,Gi].T)
clz_s_cAMP=clz_s_Gi-clz_s_Gs

clz_nons_Gs=clz_nons.loc[:,Gs].dot(pattern_df.loc[:,Gs].T)
clz_nons_Gi=clz_nons.loc[:,Gi].dot(pattern_df.loc[:,Gi].T)
clz_nons_cAMP=clz_nons_Gi-clz_nons_Gs



In [ ]:
sum(adata.obs['is_clz_selective'] ==1)

In [ ]:
mask = adata.obs['is_clz_selective'] == 1

In [ ]:
adata_clz_selective = adata[mask, :]
adata_not_clz_selective = adata[~mask, :]

GPCR_adata_clz_selective = GPCR_adata[mask, :]
GPCR_adata_not_clz_selective = GPCR_adata[~mask, :]

In [ ]:
print(adata_clz_selective.X)
print(adata_clz_selective.X.shape)
print(adata_not_clz_selective.X.shape)

In [ ]:
GPCR_list2=GPCR_adata_norm_df.columns
GPCR_type_df=GPCR_type_df[GPCR_type_df.receptor_name.isin(GPCR_list2)]

In [ ]:
print(GPCR_list2)

In [ ]:
import numpy as np

# 重複を避けるために使用するセット
unique_patterns_set = set()

# 結果を保存するための辞書
pattern_dict = {}

# 1万種類の独自の活性化パターンを生成
i = 0
while len(unique_patterns_set) < 10000:
    # ランダムな活性化パターンを生成（0はFalse、1はTrueとする）
    random_pattern = np.random.randint(2, size=len(GPCR_list2))
    # パターンを文字列に変換してハッシュ可能にする
    pattern_str = ''.join(map(str, random_pattern))

    # このパターンがまだ見つかっていない場合は保存
    if pattern_str not in unique_patterns_set:
        unique_patterns_set.add(pattern_str)
        pattern_dict[f"Pattern_{i+1}"] = {gpcr: bool(val) for gpcr, val in zip(GPCR_list2, random_pattern)}
        i += 1

# 最初の5つのパターンを出力して確認
for key in list(pattern_dict.keys())[:5]:
    print(f"{key}: {pattern_dict[key]}")
    


In [ ]:
# pattern_dictをデータフレームに変換
pattern_df = pd.DataFrame.from_dict(pattern_dict, orient='index').reset_index(drop=True)

In [ ]:
pattern_df

In [ ]:
clz_s=pd.DataFrame(GPCR_adata_clz_selective.X)
clz_s.columns=GPCR_list2

clz_nons=pd.DataFrame(GPCR_adata_not_clz_selective.X)
clz_nons.columns=GPCR_list2

In [ ]:
Gs=GPCR_type_df[GPCR_type_df.type=="Gs"]["receptor_name"].values
Gi=GPCR_type_df[GPCR_type_df.type=="Gi"]["receptor_name"].values
Gq=GPCR_type_df[GPCR_type_df.type=="Gq"]["receptor_name"].values

In [ ]:
#clz_s_Gs=(clz_s.loc[:,Gs]*pattern_df.loc[:,Gs]).sum(axis=0) 
clz_s_Gs=clz_s.loc[:,Gs].dot(pattern_df.loc[:,Gs].T)
clz_s_Gi=clz_s.loc[:,Gi].dot(pattern_df.loc[:,Gi].T)
clz_s_cAMP=clz_s_Gi-clz_s_Gs

clz_nons_Gs=clz_nons.loc[:,Gs].dot(pattern_df.loc[:,Gs].T)
clz_nons_Gi=clz_nons.loc[:,Gi].dot(pattern_df.loc[:,Gi].T)
clz_nons_cAMP=clz_nons_Gi-clz_nons_Gs

In [ ]:
from scipy.stats import ttest_ind, mannwhitneyu
def compare_and_plot(df_A, df_B, top_x, test_type='t_test', show_first_graph=True):
    # 群内平均値の群間差を計算
    mean_A = df_A.mean()
    mean_B = df_B.mean()
    diff_mean = mean_A - mean_B
    
    # 活性化パターンを群間差が大きい順にソート
    sorted_patterns = diff_mean.sort_values(ascending=False).index[:top_x]

    # 統計テスト
    test_stats = []
    p_values = []
    for pattern in sorted_patterns:
        if test_type == 't_test':
            test_stat, p_val = ttest_ind(df_A.loc[:, pattern], df_B.loc[:, pattern])
        elif test_type == 'u_test':
            test_stat, p_val = mannwhitneyu(df_A.loc[:, pattern], df_B.loc[:, pattern])
        else:
            raise ValueError("Invalid test_type. Choose either 't_test' or 'u_test'")
        
        test_stats.append(test_stat)
        p_values.append(p_val)

    # 第1のグラフ（オプション）
    if show_first_graph:
        plt.figure(figsize=(10, 6))
        plt.bar(mean_A.index, mean_A.values, alpha=0.6, label='Group A')
        plt.bar(mean_B.index, mean_B.values, alpha=0.6, label='Group B')
        plt.title('Mean Activation by Pattern')
        plt.xlabel('Pattern')
        plt.ylabel('Mean Activation')
        plt.legend()
        plt.show()

    # Sort the top_x patterns by their diff_mean, in descending order
    sorted_by_diff_mean = diff_mean[sorted_patterns].sort_values(ascending=False).index

    plt.figure(figsize=(10, 6))
    plt.scatter(range(1, top_x + 1), diff_mean[sorted_by_diff_mean], c='blue', label='Difference in Mean Activation')
    plt.xticks(range(1, top_x + 1), labels=sorted_by_diff_mean, rotation=45)
    plt.title(f'Top {top_x} Patterns with Highest Difference in Mean Activation')
    plt.xlabel('Sorted Pattern')
    plt.ylabel('Difference in Mean Activation')
    plt.legend()
    plt.show()


    return sorted_patterns, test_stats, p_values


In [ ]:
# 関数を呼び出してプロット、統計値とP値を受け取る
sorted_patterns, test_stats, p_values = compare_and_plot(clz_s_cAMP, clz_nons_cAMP,
                                                        top_x=100, test_type='t_test',show_first_graph=False)

print("Statistical values for each pattern:")
print(test_stats)

print("P-values for each pattern:")
print(p_values)

print("Top X patterns:")
print(sorted_patterns)

In [ ]:
top_pattern=pattern_df.iloc[sorted_patterns,:]

In [ ]:
def plot_heatmap_and_average(pattern_df):
    # TrueとFalseを数値に変換（True=1, False=0）
    numeric_df = pattern_df.astype(int)

    # ヒートマップを描画
    plt.figure(figsize=(10, 14))
    plt.subplot(2, 1, 1)  # 2x1のグリッドの1番目
    plt.imshow(pattern_df, cmap='RdBu_r', aspect='auto')

    # カラーバーを追加
    plt.colorbar(ticks=[0, 1], label='Boolean Value')

    # 軸のラベルとタイトルを設定
    plt.title('Pattern vs Receptor Boolean Heatmap')
    plt.xlabel('Receptors')
    plt.ylabel('Patterns')

    # x軸とy軸の目盛りを設定
    plt.xticks(range(len(pattern_df.columns)), pattern_df.columns, rotation=90)
    plt.yticks(range(len(pattern_df.index)), pattern_df.index)

    # 各受容体の論理値の平均値をプロット
    plt.subplot(2, 1, 2)  # 2x1のグリッドの2番目
    average_values = numeric_df.mean()
    plt.bar(average_values.index, average_values.values, color='green')
    plt.title('Average Boolean Value per Receptor')
    plt.xlabel('Receptor')
    plt.ylabel('Average Boolean Value')
    plt.ylim(0, 1)
    plt.xticks(rotation=90)

    plt.tight_layout()
    plt.show()


In [ ]:
# ヒートマップの描画
plot_heatmap_and_average(top_pattern)

In [ ]:
# 関数を呼び出してプロット、統計値とP値を受け取る
sorted_patterns, test_stats, p_values = compare_and_plot(clz_nons_cAMP, clz_s_cAMP,
                                                        top_x=100, test_type='t_test',show_first_graph=False)

print("Statistical values for each pattern:")
print(test_stats)

print("P-values for each pattern:")
print(p_values)

print("Top X patterns:")
print(sorted_patterns)

top_pattern=pattern_df.iloc[sorted_patterns,:]
plot_heatmap_and_average(top_pattern)

In [ ]:
important_receptor=["HTR1B","HRH3","CHRM2","ADRB2"]